In [2]:
import pandas as pd
import numpy as np
import joblib  # Dùng để lưu model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns # Dùng để vẽ ma trận nhầm lẫn
import matplotlib.pyplot as plt

print("Tất cả thư viện đã được nạp!")

Tất cả thư viện đã được nạp!


In [3]:
# File paths
TRAIN_FILE = '../1_data/processed/train_landmarks_augmented.csv'
VALID_FILE = '../1_data/processed/valid_landmarks.csv'
TEST_FILE = '../1_data/processed/test_landmarks.csv'

# Read data
df_train = pd.read_csv(TRAIN_FILE)
df_valid = pd.read_csv(VALID_FILE)
df_test = pd.read_csv(TEST_FILE)
labels_to_remove = ['DD'] 

df_train = df_train[~df_train['label'].isin(labels_to_remove)]
df_valid = df_valid[~df_valid['label'].isin(labels_to_remove)]
df_test = df_test[~df_test['label'].isin(labels_to_remove)]

print(f"Load {len(df_train)} row Train.")
print(f"Load {len(df_valid)} row Valid.")
print(f"Load {len(df_test)} row Test.")

# Show first 5 rows of training data
df_train.head()

Load 21418 row Train.
Load 1484 row Valid.
Load 682 row Test.


,label,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
0,A,0.0,0.0,0.0,0.074581,-0.073236,-0.021343,0.119197,-0.180595,-0.027051,...,-0.020520,-0.060482,-0.280165,-0.045432,-0.037218,-0.223027,-0.034378,-0.029876,-0.169899,-0.017204
1,A,0.0,0.0,0.0,0.074581,-0.073236,-0.021343,0.119197,-0.180595,-0.027051,...,-0.020520,-0.060482,-0.280165,-0.045432,-0.037218,-0.223027,-0.034378,-0.029876,-0.169899,-0.017204
2,A,0.0,0.0,0.0,0.078033,-0.069783,-0.021800,0.126030,-0.169685,-0.027774,...,-0.019657,-0.070106,-0.259200,-0.044702,-0.042814,-0.210913,-0.036414,-0.029962,-0.161349,-0.021522
3,A,0.0,0.0,0.0,0.071760,-0.087289,-0.021118,0.110809,-0.202221,-0.026705,...,-0.019010,-0.077575,-0.284882,-0.045768,-0.048861,-0.228968,-0.034834,-0.035993,-0.175218,-0.016719
4,A,0.0,0.0,0.0,0.085723,-0.059413,-0.022042,0.143285,-0.163324,-0.027057,...,-0.020827,-0.034273,-0.295545,-0.048744,-0.014472,-0.239036,-0.041396,-0.006546,-0.182444,-0.025817


In [3]:
# Tách cột 'label' ra làm 'y' (đáp án)
y_train = df_train['label']
y_valid = df_valid['label']
y_test = df_test['label']

# Bỏ cột 'label' để lấy 'X' (dữ liệu đầu vào)
X_train = df_train.drop('label', axis=1)
X_valid = df_valid.drop('label', axis=1)
X_test = df_test.drop('label', axis=1)

print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước X_valid: {X_valid.shape}")
print(f"Kích thước X_test: {X_test.shape}")

Kích thước X_train: (23014, 63)
Kích thước X_valid: (1582, 63)
Kích thước X_test: (743, 63)


In [4]:
# Tạo một 'cỗ máy' chuẩn hóa
scaler = StandardScaler()

# B1: "Học" thang đo từ DỮ LIỆU TRAIN
scaler.fit(X_train)

# B2: "Áp dụng" thang đo đó cho cả 3 tập
X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

print("Đã chuẩn hóa (scale) dữ liệu.")

Đã chuẩn hóa (scale) dữ liệu.


In [5]:
# Khởi tạo model
# n_estimators=100: Dùng 100 "cây" quyết định
# random_state=42: Để đảm bảo kết quả có thể tái lập
print("Bắt đầu huấn luyện RandomForest...")

model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# "Dạy" cho model bằng dữ liệu train đã chuẩn hóa
model.fit(X_train_scaled, y_train)

print("--- HUẤN LUYỆN HOÀN TẤT! ---")

Bắt đầu huấn luyện RandomForest...
--- HUẤN LUYỆN HOÀN TẤT! ---


In [6]:
# Yêu cầu model dự đoán kết quả cho tập valid
valid_preds = model.predict(X_valid_scaled)

# Tính toán độ chính xác
valid_accuracy = accuracy_score(y_valid, valid_preds)

print(f"--- KẾT QUẢ TRÊN TẬP VALID ---")
print(f"Độ chính xác (Accuracy): {valid_accuracy * 100:.2f}%")

# In ra báo cáo chi tiết (Precision, Recall, F1-score)
print("\nBáo cáo chi tiết (Classification Report):\n")
print(classification_report(y_valid, valid_preds))

--- KẾT QUẢ TRÊN TẬP VALID ---
Độ chính xác (Accuracy): 99.37%

Báo cáo chi tiết (Classification Report):

              precision    recall  f1-score   support

           A       1.00      1.00      1.00        68
           B       1.00      1.00      1.00        73
           C       0.99      1.00      0.99        68
           D       0.90      0.79      0.84        24
          DD       0.95      0.98      0.96        98
           E       1.00      1.00      1.00        54
           G       1.00      1.00      1.00        56
           H       1.00      1.00      1.00        63
           I       1.00      1.00      1.00        82
           K       1.00      1.00      1.00        68
           L       1.00      1.00      1.00        53
           M       1.00      1.00      1.00        49
           N       1.00      1.00      1.00        61
           O       0.99      0.99      0.99        86
           P       1.00      1.00      1.00        90
           Q       1.00     

In [7]:
# Yêu cầu model dự đoán kết quả cho tập test
test_preds = model.predict(X_test_scaled)

# Tính toán độ chính xác
test_accuracy = accuracy_score(y_test, test_preds)

print(f"--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST ---")
print(f"Độ chính xác (Accuracy): {test_accuracy * 100:.2f}%")

--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST ---
Độ chính xác (Accuracy): 99.06%


In [8]:
# Tạo thư mục nếu chưa có
import os
os.makedirs('../4_models/', exist_ok=True)

# Đặt tên file
MODEL_PATH = '../4_models/sign_language_model.pkl'
SCALER_PATH = '../4_models/scaler.pkl'

# Lưu file
joblib.dump(model, MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)

print(f"Đã lưu model vào: {MODEL_PATH}")
print(f"Đã lưu scaler vào: {SCALER_PATH}")

Đã lưu model vào: ../4_models/sign_language_model.pkl
Đã lưu scaler vào: ../4_models/scaler.pkl
